In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import os
import nltk
import numpy as np
import networkx as nx
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def preprocess_java_file(file_path):
    with open(file_path, 'r') as file:
        java_code = file.read()
    # Tokenize the Java code
    tokens = word_tokenize(java_code)
    return tokens

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

Infer to vector

In [ ]:

# Try using different encodings
try:
    model = gensim.models.Doc2Vec.load("/content/drive/MyDrive/ATBM/doc2vec/doc2vec_model", mmap='r')
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)


Model loaded successfully.


In [ ]:
c

In [ ]:
#old
# gexf_folder = '/content/drive/MyDrive/ATBM/Tung/fcg_benign'
# java_folder = '/content/drive/MyDrive/ATBM/Tung/java_benign'

gexf_folder = '/content/drive/MyDrive/ATBM/Tung/BTL_ATBM_MINH/op_fcg_benign_final'
java_folder = '/content/drive/MyDrive/ATBM/Tung/BTL_ATBM_MINH/op_java_benign_final'
output_feat_folder = '/content/drive/MyDrive/ATBM/Tung/vec_benign'

print(len(os.listdir(gexf_folder)))
print(len(os.listdir(java_folder)))
print(len(os.listdir(output_feat_folder)))

3256
3256
897


In [ ]:
# output_feat_folder = ["fhiasdhfisadf()", "ifbasidbf"]
for filename in os.listdir(output_feat_folder):
  if "(" in filename:
    print(filename)
for filename in os.listdir(output_feat_folder):
  if "(" in filename:
    os.remove(output_feat_folder+"/"+filename)

## Count fcg file that has java

In [ ]:
# def get_fcgs_has_java(gexf_folder, java_folder):
#   cnt = 0
#   a = []
#   for gexf_filename in os.listdir(gexf_folder):
#     if gexf_filename.endswith('.gexf'):
#       apk_id = gexf_filename[:-5]
#       java_filepath = os.path.join(java_folder, f'{apk_id}.txt')
#       if(f'{apk_id}.txt' in os.listdir(java_folder)):
#         cnt+=1
#         a.append(apk_id)
#   return a
# fcgs_has_java = get_fcgs_has_java(gexf_folder, java_folder)
# print(len(fcgs_has_java))

# left = []
# for x in fcgs_has_java:
#   if(f'{x}.txt') not in output_feat_folder:
#     left.append(x)
# print(len(left))

## Count file in folder

In [ ]:
import os

def count_files(folder_path):
  try:
    entries = os.scandir(folder_path)
    files = [entry for entry in entries if entry.is_file()]
    return len(files)
  except FileNotFoundError:
    print(f"Folder not found: {folder_path}")
    return 0  # Return 0 if folder doesn't exist

# Example usage (replace with your actual folder path)
print(f"Number of files in folder {gexf_folder.split('/')[-1]}: {count_files(gexf_folder)}")
print(f"Number of files in folder {java_folder.split('/')[-1]}: {count_files(java_folder)}")
print(f"Number of files in folder {output_feat_folder.split('/')[-1]}: {count_files(output_feat_folder)}")
# print(f"Number of files in fcgs_has_java: {len(fcgs_has_java)}")
#828




Number of files in folder op_fcg_benign_final: 3256
Number of files in folder op_java_benign_final: 3256
Number of files in folder vec_benign: 887


In [ ]:
import os

gexf_files = [file[:-5] for file in os.listdir(gexf_folder) if file.endswith('.gexf')]
txt_files = [file[:-4] for file in os.listdir(output_feat_folder) if file.endswith('.txt')]

gexf_files_set = set(gexf_files)
txt_files_set = set(txt_files)

unique_gexf_files = gexf_files_set - txt_files_set
print(len(unique_gexf_files))
print('CE5119808A8F2138609B5322FBCBAD8DA8E9ADEEFE36004BFE67214C60DD759D' in unique_gexf_files)

2370
False


In [ ]:
import os
import re
import networkx as nx
from gensim.models import Doc2Vec

def extract_and_embed_functions(gexf_folder, java_folder, doc2vec_model, output_feat_folder):

  # Load the Doc2Vec model
  model = doc2vec_model
  # Iterate over GEXF files
  for gexf_filename in os.listdir(gexf_folder):
    if gexf_filename.endswith('.gexf'):
      apk_id = gexf_filename[:-5]  # Extract APK ID from filename (assuming format)
      if(apk_id not in fcgs_has_java):
        continue
      if(apk_id not in unique_gexf_files):
        print("HAS EMBEDDED!")
        continue
      # Construct file paths
      gexf_filepath = os.path.join(gexf_folder, gexf_filename)
      java_filepath = os.path.join(java_folder, f'{apk_id}.txt')
      output_feat_folder_filepath = os.path.join(output_feat_folder, f'{apk_id}.txt')

      # Load the GEXF graph
      graph = nx.read_gexf(gexf_filepath)
      function_names = [node for node in graph.nodes()]

      # Read Java content
      try:
          with open(java_filepath, 'r') as file:
              java_content = file.read()
      except FileNotFoundError:
          print(f"Java file not found: {java_filepath}")
          continue  # Skip to next iteration if Java file is missing

      print("Running " + gexf_filename)
      # Extract and embed function content
      fc_str = []
      for function_name in function_names:
          parameters, body = extract_function_content(java_content, function_name)
          if parameters is not None and body is not None:
              processed_fc = f'{function_name}({parameters})' + '{' + f'{body}' + '}'
          else:
              processed_fc = f'{function_name}({parameters})' + '{}'
          fc_str.append(processed_fc)

      # Preprocess if necessary (using NLTK for example)
      from nltk import word_tokenize  # Import NLTK library for tokenization

      preprocessed_strings = []
      for string in fc_str:
          tokens = word_tokenize(string)
          preprocessed_strings.append(tokens)

      # Embed each string (using preprocessed strings)
      string_vectors = []
      for string in preprocessed_strings:
          vector = model.infer_vector(string)  # Use infer_vector for unseen data
          string_vectors.append(vector)

      # Save FEATURE MATRIX to a text file (one vector per line)
      try:
          with open(output_feat_folder_filepath, 'w') as output_file:
              print(output_feat_folder_filepath)
              for vector in string_vectors:
                  output_file.write(str(vector) + '\n')  # Write each vector on a new line
          print("DONE "+gexf_filename)
      except IOError:
          print(f"Error saving output to: {output_feat_folder_filepath}")

extract_and_embed_functions(gexf_folder, java_folder, model, output_feat_folder)

HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EMBEDDED!
HAS EM

ZIP

In [ ]:
import shutil

# Specify the source folder and output zip file
folder = 'benign_vec'  # Replace with the path and name of the zip file (without .zip)

# Create a zip file from the folder
shutil.make_archive(folder, 'zip', folder)

print(f'Folder {folder} has been zipped to {folder}.zip')